In [7]:
import re
import numpy as np
import pandas as pd
from scipy import stats

# Offensive


In [3]:
offensive = pd.read_table("offensive_results_w.tsv")

In [36]:
o_error_base_flant5xxl = -(offensive.loc[:, 'label':'asian_score']).sub(offensive['flant5_xxl_base'], axis=0)
o_error_base_flant5xxl['model'] = 'flan_t5_xxl'
o_error_base_flan_ul2 = -(offensive.loc[:, 'label':'asian_score']).sub(offensive['flan_ul2_base'], axis=0)
o_error_base_flan_ul2['model'] = 'flan_ul2'
o_error_base_tulu2_7b = -(offensive.loc[:, 'label':'asian_score']).sub(offensive['tulu2_7b_base'], axis=0)
o_error_base_tulu2_7b['model'] = 'tulu2_7b'
o_error_base_tulu2_13b = -(offensive.loc[:, 'label':'asian_score']).sub(offensive['tulu2_13b_base'], axis=0)
o_error_base_tulu2_13b['model'] = 'tulu2_13b'
o_error_base_gpt35 = -(offensive.loc[:, 'label':'asian_score']).sub(offensive['gpt35_base'], axis=0)
o_error_base_gpt35['model'] = 'gpt35'
o_error_base_gpt4 = -(offensive.loc[:, 'label':'asian_score']).sub(offensive['gpt4_base'], axis=0)
o_error_base_gpt4['model'] = 'gpt4'

o_error_base = (pd.concat([o_error_base_flant5xxl, o_error_base_flan_ul2, o_error_base_tulu2_7b, o_error_base_tulu2_13b,
                          o_error_base_gpt35, o_error_base_gpt4],
                          ignore_index=True)
                .rename(columns={'label':'overall_score'}))

In [37]:
o_error_identity_flant5xxl = -(offensive.loc[:, 'label':'asian_score']-(offensive.loc[:,'flant5_xxl_base':'flant5_xxl_asian'].values))
o_error_identity_flant5xxl['model'] = 'flan_t5_xxl'
o_error_identity_flan_ul2 = -(offensive.loc[:, 'label':'asian_score']-(offensive.loc[:, 'flan_ul2_base':'flan_ul2_asian'].values))
o_error_identity_flan_ul2['model'] = 'flan_ul2'
o_error_identity_tulu2_7b = -(offensive.loc[:, 'label':'asian_score']-(offensive.loc[:, 'tulu2_7b_base':'tulu2_7b_asian'].values))
o_error_identity_tulu2_7b['model'] = 'tulu2_7b'
o_error_identity_tulu2_13b = -(offensive.loc[:, 'label':'asian_score']-(offensive.loc[:, 'tulu2_13b_base':'tulu2_13b_asian'].values))
o_error_identity_tulu2_13b['model'] = 'tulu2_13b'
o_error_identity_gpt35 = -(offensive.loc[:, 'label':'asian_score']-(offensive.loc[:, 'gpt35_base':'gpt35_asian'].values))
o_error_identity_gpt35['model'] = 'gpt35'
o_error_identity_gpt4 = -(offensive.loc[:, 'label':'asian_score']-(offensive.loc[:, 'gpt4_base':'gpt4_asian'].values))
o_error_identity_gpt4['model'] = 'gpt4'

o_error_identity = (pd.concat([o_error_identity_flant5xxl, o_error_identity_flan_ul2, o_error_identity_tulu2_7b, 
                               o_error_identity_tulu2_13b, o_error_identity_gpt35, o_error_identity_gpt4],
                               ignore_index=True)
                .rename(columns={'label':'overall_score'}))

## Positional Difference

### Base

base predictions vs. identity labels

In [38]:
o_error_base.groupby(['model']).agg([np.mean, np.std]).round(3)

overall_score        man_score        woman_score         \
                     mean    std      mean    std        mean    std   
model                                                                  
flan_t5_xxl         0.376  0.924     0.376  1.033       0.363  1.018   
flan_ul2            1.064  1.166     1.064  1.256       1.045  1.238   
gpt35               0.438  0.647     0.435  0.784       0.425  0.788   
gpt4                0.217  0.980     0.216  1.089       0.199  1.052   
tulu2_13b           1.372  0.719     1.369  0.835       1.359  0.856   
tulu2_7b            1.910  0.966     1.916  1.071       1.898  1.063   

            white_score        black_score        asian_score         
                   mean    std        mean    std        mean    std  
model                                                                 
flan_t5_xxl       0.391  0.954       0.190  1.343       0.351  1.340  
flan_ul2          1.079  1.186       0.882  1.534       1.017  1.458  
gpt35             0.454  0.678       0.227  1.196       0.425  1.097  
gpt4              0.233  0.989       0.012  1.395       0.202  1.388  
tulu2_13b         1.387  0.747       1.173  1.236       1.353  1.157  
tulu2_7b          1.927  0.979       1.694  1.422       1.895  1.357

### Idendity

identity predictions vs. identity labels

In [39]:
o_error_identity.groupby(['model']).agg([np.mean, np.std]).round(3)

overall_score        man_score        woman_score         \
                     mean    std      mean    std        mean    std   
model                                                                  
flan_t5_xxl         0.376  0.924     0.425  1.098       0.453  1.096   
flan_ul2            1.064  1.166     1.146  1.247       1.090  1.296   
gpt35               0.438  0.647     0.275  0.798       0.266  0.811   
gpt4                0.217  0.980    -0.326  0.944      -0.188  1.024   
tulu2_13b           1.372  0.719     1.151  0.845       1.031  0.876   
tulu2_7b            1.910  0.966     1.864  0.979       1.856  0.970   

            white_score        black_score        asian_score         
                   mean    std        mean    std        mean    std  
model                                                                 
flan_t5_xxl       0.105  0.944       0.107  1.410       0.316  1.383  
flan_ul2          0.837  1.287       0.272  1.692       0.090  1.537  
gpt35             0.019  0.726      -0.230  1.163       0.070  1.092  
gpt4             -0.578  0.785      -0.849  1.303      -0.666  1.187  
tulu2_13b         1.217  0.734       1.074  1.318       1.093  1.192  
tulu2_7b          1.754  0.901       1.597  1.335       1.826  1.225

## Test the difference

### Base, Distance

an indirect statistical test to justify our findings in Fig 2.

(We conduct post hoc t-tests with Bonferoni correction to ensure that there are between-group differences in the distances between base predictions and identity labels. In our main findings, we use the gaps in correlation coefficients to illustrate which group the base predictions are closer to.)

In [44]:
models = o_error_base['model'].unique()
results = {'model':[], 'pair':[], 't_stat':[], 'p_value':[]}

for i in models:
    df = o_error_base[o_error_base['model']==i]
    g = stats.ttest_rel(df['man_score'], df['woman_score'], nan_policy="omit")
    e1 = stats.ttest_rel(df['white_score'], df['black_score'], nan_policy="omit")
    e2 = stats.ttest_rel(df['white_score'], df['asian_score'], nan_policy="omit")
    e3 = stats.ttest_rel(df['black_score'], df['asian_score'], nan_policy="omit")
    results['model'].extend([i,i,i,i])
    results['pair'].extend(['man_woman', 'white_black', 'white_asian', 'black_asian'])
    results['t_stat'].extend([g[0], e1[0], e2[0], e3[0]])
    results['p_value'].extend([g[1], e1[1], e2[1], e3[1]])

o_results_base = pd.DataFrame.from_dict(results, orient="columns")

In [45]:
o_results_base.pivot(index='model', columns='pair', values=['t_stat', 'p_value']).round(3)

t_stat                                       p_value  \
pair        black_asian man_woman white_asian white_black black_asian   
model                                                                   
flan_t5_xxl      -1.136     0.692       0.973       6.142       0.256   
flan_ul2         -1.136     0.692       0.973       6.142       0.256   
gpt35            -1.176     0.607       0.911       6.139       0.240   
gpt4             -1.136     0.692       0.973       6.142       0.256   
tulu2_13b        -0.809     0.649       1.039       5.964       0.419   
tulu2_7b         -1.150     0.790       1.155       5.673       0.251   

                                               
pair        man_woman white_asian white_black  
model                                          
flan_t5_xxl     0.489       0.331         0.0  
flan_ul2        0.489       0.331         0.0  
gpt35           0.544       0.363         0.0  
gpt4            0.489       0.331         0.0  
tulu2_13b       0.516       0.299         0.0  
tulu2_7b        0.430       0.248         0.0

### Identity

#### Distance

show that adding identity tokens has influence on the alignment of models' predictions to subgroup labels.

In [47]:
models = o_error_identity['model'].unique()
results = {'model':[], 'identity':[], 't_stat':[], 'p_value':[]}

for i in models:
    df_base = o_error_base[o_error_base['model']==i]
    df_identity = o_error_identity[o_error_identity['model']==i]
    man = stats.ttest_rel(df_base['man_score'], df_identity['man_score'], nan_policy="omit")
    woman = stats.ttest_rel(df_base['woman_score'], df_identity['woman_score'], nan_policy="omit")
    white = stats.ttest_rel(df_base['white_score'], df_identity['white_score'], nan_policy="omit")
    black = stats.ttest_rel(df_base['black_score'], df_identity['black_score'], nan_policy="omit")
    asian = stats.ttest_rel(df_base['asian_score'], df_identity['asian_score'], nan_policy="omit")
    results['model'].extend([i,i,i,i,i])
    results['identity'].extend(['man', 'woman', 'white', 'black', 'asian'])
    results['t_stat'].extend([man[0], woman[0], white[0], black[0], asian[0]])
    results['p_value'].extend([man[1], woman[1], white[1], black[1], asian[1]])

o_results_identity = pd.DataFrame.from_dict(results, orient="columns")

In [49]:
o_results_identity.pivot(index='model', columns='identity', values=['t_stat', 'p_value']).round(3)

t_stat                                 p_value                \
identity      asian   black     man   white   woman   asian  black    man   
model                                                                       
flan_t5_xxl   1.904   3.710  -4.409  17.476  -7.420   0.057  0.000  0.000   
flan_ul2     19.024  15.933  -5.200  10.904  -2.656   0.000  0.000  0.000   
gpt35        10.536  12.181  13.467  25.199   9.783   0.000  0.000  0.000   
gpt4         18.470  22.211  25.987  28.762  18.038   0.000  0.000  0.000   
tulu2_13b     9.260   3.231  16.159  12.226  21.584   0.000  0.001  0.000   
tulu2_7b      1.948   2.885   2.626   8.273   1.750   0.052  0.004  0.009   

                          
identity    white  woman  
model                     
flan_t5_xxl   0.0  0.000  
flan_ul2      0.0  0.008  
gpt35         0.0  0.000  
gpt4          0.0  0.000  
tulu2_13b     0.0  0.000  
tulu2_7b      0.0  0.080

#### Prediction

show that there is significant difference of models' behavior w/o identity tokens.

In [75]:
o_pred_error = (offensive.loc[:, 'flant5_xxl_base':'gpt4_asian'].reset_index()
                .melt(id_vars='index', value_name='pred', var_name='var'))
o_pred_error['model'] = o_pred_error['var'].apply(lambda x: re.split(r'_[a-z]+$', x)[0])
o_pred_error['identity'] = o_pred_error['var'].apply(lambda x: re.findall(r'[a-z]+$', x)[0])
del o_pred_error['var']

In [76]:
models = o_pred_error['model'].unique()
identities = o_pred_error['identity'].unique()
results = {'model':[], 'identity':[], 't_stat':[], 'p_value':[]}

for i in models:
    df = o_pred_error[o_pred_error['model']==i]
    df_base = df[df['identity']=='base']
    for j in identities:
        if j == "base":
            continue
        df_identity = df[df['identity']==j]
        ttest = stats.ttest_rel(df_base['pred'], df_identity['pred'], nan_policy='omit')
        results['model'].append(i)
        results['identity'].append(j)
        results['t_stat'].append(ttest[0])
        results['p_value'].append(ttest[1])

o_results_pred = pd.DataFrame.from_dict(results, orient="columns")

In [77]:
o_results_pred.pivot(index='model', columns='identity', values=['t_stat', 'p_value']).round(3)

t_stat                                 p_value                \
identity     asian   black  female    male   white   asian  black female   
model                                                                      
flan_ul2    29.009  18.731  -2.599  -5.316  10.904   0.000  0.000  0.009   
flant5_xxl   0.923   4.469  -7.420  -4.569  17.476   0.356  0.000  0.000   
gpt35       14.713  14.879   9.839  13.710  25.199   0.000  0.000  0.000   
gpt4        26.593  27.069  18.109  26.157  28.762   0.000  0.000  0.000   
tulu2_13b   12.461   3.293  21.661  16.262  12.226   0.000  0.001  0.000   
tulu2_7b     2.429   3.784   1.791   2.518   8.273   0.015  0.000  0.074   

                         
identity     male white  
model                    
flan_ul2    0.000   0.0  
flant5_xxl  0.000   0.0  
gpt35       0.000   0.0  
gpt4        0.000   0.0  
tulu2_13b   0.000   0.0  
tulu2_7b    0.012   0.0

# Politeness

In [30]:
polite = pd.read_table("polite_results_w.tsv")

In [40]:
p_error_base_flant5xxl = -(polite.loc[:, 'label':'asian_score']).sub(polite['flant5_xxl_base'], axis=0)
p_error_base_flant5xxl['model'] = 'flan_t5_xxl'
p_error_base_flan_ul2 = -(polite.loc[:, 'label':'asian_score']).sub(polite['flan_ul2_base'], axis=0)
p_error_base_flan_ul2['model'] = 'flan_ul2'
p_error_base_tulu2_7b = -(polite.loc[:, 'label':'asian_score']).sub(polite['tulu2_7b_base'], axis=0)
p_error_base_tulu2_7b['model'] = 'tulu2_7b'
p_error_base_tulu2_13b = -(polite.loc[:, 'label':'asian_score']).sub(polite['tulu2_13b_base'], axis=0)
p_error_base_tulu2_13b['model'] = 'tulu2_13b'
p_error_base_gpt35 = -(polite.loc[:, 'label':'asian_score']).sub(polite['gpt35_base'], axis=0)
p_error_base_gpt35['model'] = 'gpt35'
p_error_base_gpt4 = -(polite.loc[:, 'label':'asian_score']).sub(polite['gpt4_base'], axis=0)
p_error_base_gpt4['model'] = 'gpt4'

p_error_base = (pd.concat([p_error_base_flant5xxl, p_error_base_flan_ul2, p_error_base_tulu2_7b, p_error_base_tulu2_13b,
                          p_error_base_gpt35, p_error_base_gpt4],
                          ignore_index=True)
                .rename(columns={'label':'overall_score'}))

In [41]:
p_error_identity_flant5xxl = -(polite.loc[:, 'label':'asian_score']-(polite.loc[:,'flant5_xxl_base':'flant5_xxl_asian'].values))
p_error_identity_flant5xxl['model'] = 'flan_t5_xxl'
p_error_identity_flan_ul2 = -(polite.loc[:, 'label':'asian_score']-(polite.loc[:, 'flan_ul2_base':'flan_ul2_asian'].values))
p_error_identity_flan_ul2['model'] = 'flan_ul2'
p_error_identity_tulu2_7b = -(polite.loc[:, 'label':'asian_score']-(polite.loc[:, 'tulu2_7b_base':'tulu2_7b_asian'].values))
p_error_identity_tulu2_7b['model'] = 'tulu2_7b'
p_error_identity_tulu2_13b = -(polite.loc[:, 'label':'asian_score']-(polite.loc[:, 'tulu2_13b_base':'tulu2_13b_asian'].values))
p_error_identity_tulu2_13b['model'] = 'tulu2_13b'
p_error_identity_gpt35 = -(polite.loc[:, 'label':'asian_score']-(polite.loc[:, 'gpt35_base':'gpt35_asian'].values))
p_error_identity_gpt35['model'] = 'gpt35'
p_error_identity_gpt4 = -(polite.loc[:, 'label':'asian_score']-(polite.loc[:, 'gpt4_base':'gpt4_asian'].values))
p_error_identity_gpt4['model'] = 'gpt4'

p_error_identity = (pd.concat([p_error_identity_flant5xxl, p_error_identity_flan_ul2, p_error_identity_tulu2_7b,
                               p_error_identity_tulu2_13b, p_error_identity_gpt35, p_error_identity_gpt4],
                               ignore_index=True)
                .rename(columns={'label':'overall_score'}))

## Positional Difference

### Base

base predictions vs. identity labels

In [42]:
p_error_base.groupby(['model']).agg([np.mean, np.std]).round(3)

overall_score        man_score        woman_score         \
                     mean    std      mean    std        mean    std   
model                                                                  
flan_t5_xxl         0.210  0.891     0.196  1.030       0.205  0.971   
flan_ul2            0.597  0.901     0.585  1.021       0.594  1.001   
gpt35              -0.041  0.629    -0.057  0.784      -0.045  0.767   
gpt4                0.605  0.719     0.590  0.889       0.601  0.816   
tulu2_13b           0.048  0.764     0.032  0.908       0.046  0.882   
tulu2_7b            0.024  0.816     0.012  0.944       0.022  0.935   

            white_score        black_score        asian_score         
                   mean    std        mean    std        mean    std  
model                                                                 
flan_t5_xxl       0.211  0.917       0.116  1.232       0.309  1.195  
flan_ul2          0.598  0.929       0.487  1.237       0.705  1.215  
gpt35            -0.040  0.673      -0.147  1.050       0.058  1.004  
gpt4              0.606  0.757       0.495  1.114       0.703  1.061  
tulu2_13b         0.048  0.801      -0.039  1.145       0.160  1.081  
tulu2_7b          0.025  0.844      -0.068  1.161       0.137  1.160

### Idendity

identity predictions vs. identity labels

In [43]:
p_error_identity.groupby(['model']).agg([np.mean, np.std]).round(3)

overall_score        man_score        woman_score         \
                     mean    std      mean    std        mean    std   
model                                                                  
flan_t5_xxl         0.210  0.891     0.204  1.062       0.188  0.999   
flan_ul2            0.597  0.901     0.669  1.003       0.570  1.011   
gpt35              -0.041  0.629     0.013  0.776      -0.064  0.774   
gpt4                0.605  0.719     0.686  0.862       0.592  0.805   
tulu2_13b           0.048  0.764     0.004  0.893      -0.073  0.896   
tulu2_7b            0.024  0.816     0.128  0.904       0.145  0.911   

            white_score        black_score        asian_score         
                   mean    std        mean    std        mean    std  
model                                                                 
flan_t5_xxl       0.291  0.939       0.186  1.269       0.215  1.195  
flan_ul2          0.589  0.929       0.379  1.229       0.505  1.211  
gpt35             0.050  0.691      -0.133  1.072       0.024  1.034  
gpt4              0.706  0.734       0.792  1.123       0.538  1.041  
tulu2_13b         0.214  0.816       0.176  1.151      -0.010  1.064  
tulu2_7b          0.248  0.781      -0.023  1.095       0.243  1.123

## Test the difference

### Base, Distance

an indirect statistical test to justify our findings in Fig 2.

(We conduct post hoc t-tests with Bonferoni correction to ensure that there are between-group differences in the distances between base predictions and identity labels. In our main findings, we use the gaps in correlation coefficients to illustrate which group the base predictions are closer to.)

In [50]:
models = p_error_base['model'].unique()
results = {'model':[], 'pair':[], 't_stat':[], 'p_value':[]}

for i in models:
    df = p_error_base[p_error_base['model']==i]
    g = stats.ttest_rel(df['man_score'], df['woman_score'], nan_policy="omit")
    e1 = stats.ttest_rel(df['white_score'], df['black_score'], nan_policy="omit")
    e2 = stats.ttest_rel(df['white_score'], df['asian_score'], nan_policy="omit")
    e3 = stats.ttest_rel(df['black_score'], df['asian_score'], nan_policy="omit")
    results['model'].extend([i,i,i,i])
    results['pair'].extend(['man_woman', 'white_black', 'white_asian', 'black_asian'])
    results['t_stat'].extend([g[0], e1[0], e2[0], e3[0]])
    results['p_value'].extend([g[1], e1[1], e2[1], e3[1]])

p_results_base = pd.DataFrame.from_dict(results, orient="columns")

In [52]:
p_results_base.pivot(index='model', columns='pair', values=['t_stat', 'p_value']).round(3)

t_stat                                       p_value  \
pair        black_asian man_woman white_asian white_black black_asian   
model                                                                   
flan_t5_xxl      -4.344    -0.765      -3.828       4.614         0.0   
flan_ul2         -4.344    -0.760      -3.840       4.614         0.0   
gpt35            -4.344    -0.815      -3.828       4.658         0.0   
gpt4             -4.344    -0.765      -3.828       4.614         0.0   
tulu2_13b        -4.097    -0.964      -3.849       4.293         0.0   
tulu2_7b         -4.245    -0.519      -3.740       4.825         0.0   

                                               
pair        man_woman white_asian white_black  
model                                          
flan_t5_xxl     0.444         0.0         0.0  
flan_ul2        0.447         0.0         0.0  
gpt35           0.415         0.0         0.0  
gpt4            0.444         0.0         0.0  
tulu2_13b       0.335         0.0         0.0  
tulu2_7b        0.604         0.0         0.0

### Identity

#### Distance

show that adding identity tokens has influence on the alignment of models' predictions to subgroup labels.

In [51]:
models = p_error_identity['model'].unique()
results = {'model':[], 'identity':[], 't_stat':[], 'p_value':[]}

for i in models:
    df_base = p_error_base[p_error_base['model']==i]
    df_identity = p_error_identity[p_error_identity['model']==i]
    man = stats.ttest_rel(df_base['man_score'], df_identity['man_score'], nan_policy="omit")
    woman = stats.ttest_rel(df_base['woman_score'], df_identity['woman_score'], nan_policy="omit")
    white = stats.ttest_rel(df_base['white_score'], df_identity['white_score'], nan_policy="omit")
    black = stats.ttest_rel(df_base['black_score'], df_identity['black_score'], nan_policy="omit")
    asian = stats.ttest_rel(df_base['asian_score'], df_identity['asian_score'], nan_policy="omit")
    results['model'].extend([i,i,i,i,i])
    results['identity'].extend(['man', 'woman', 'white', 'black', 'asian'])
    results['t_stat'].extend([man[0], woman[0], white[0], black[0], asian[0]])
    results['p_value'].extend([man[1], woman[1], white[1], black[1], asian[1]])

p_results_identity = pd.DataFrame.from_dict(results, orient="columns")

In [53]:
p_results_identity.pivot(index='model', columns='identity', values=['t_stat', 'p_value']).round(3)

t_stat                                 p_value                \
identity      asian   black     man   white   woman   asian  black    man   
model                                                                       
flan_t5_xxl   7.566  -7.730  -1.342 -12.974   2.913   0.000  0.000  0.180   
flan_ul2     12.804   9.101 -11.513   1.568   3.387   0.000  0.000  0.000   
gpt35         2.117  -1.334 -12.440 -12.526   3.550   0.034  0.182  0.000   
gpt4         13.297 -24.788 -15.610 -15.938   1.540   0.000  0.000  0.000   
tulu2_13b    10.572 -17.166   2.844 -20.446  13.265   0.000  0.000  0.004   
tulu2_7b     -6.224  -4.682 -15.074 -23.456 -15.345   0.000  0.000  0.000   

                           
identity     white  woman  
model                      
flan_t5_xxl  0.000  0.004  
flan_ul2     0.117  0.001  
gpt35        0.000  0.000  
gpt4         0.000  0.124  
tulu2_13b    0.000  0.000  
tulu2_7b     0.000  0.000

#### Prediction

show that there is significant difference of models' behavior w/o identity tokens.

In [68]:
p_pred_error = (polite.loc[:, 'flant5_xxl_base':'gpt4_asian'].reset_index()
                .melt(id_vars='index', value_name='pred', var_name='var'))
p_pred_error['model'] = p_pred_error['var'].apply(lambda x: re.split(r'_[a-z]+$', x)[0])
p_pred_error['identity'] = p_pred_error['var'].apply(lambda x: re.findall(r'[a-z]+$', x)[0])
del p_pred_error['var']

In [73]:
models = p_pred_error['model'].unique()
identities = p_pred_error['identity'].unique()
results = {'model':[], 'identity':[], 't_stat':[], 'p_value':[]}

for i in models:
    df = p_pred_error[p_pred_error['model']==i]
    df_base = df[df['identity']=='base']
    for j in identities:
        if j == "base":
            continue
        df_identity = df[df['identity']==j]
        ttest = stats.ttest_rel(df_base['pred'], df_identity['pred'], nan_policy='omit')
        results['model'].append(i)
        results['identity'].append(j)
        results['t_stat'].append(ttest[0])
        results['p_value'].append(ttest[1])

p_results_pred = pd.DataFrame.from_dict(results, orient="columns")

In [74]:
p_results_pred.pivot(index='model', columns='identity', values=['t_stat', 'p_value']).round(3)

t_stat                                 p_value                \
identity     asian   black  female    male   white   asian  black female   
model                                                                      
flan_ul2    21.871  12.293   3.417 -11.507   1.568     0.0  0.000  0.001   
flant5_xxl  12.516 -10.167   3.031  -1.518 -12.974     0.0  0.000  0.002   
gpt35        4.304  -1.874   3.431 -12.340 -12.557     0.0  0.061  0.001   
gpt4        21.239 -32.929   1.582 -15.636 -15.938     0.0  0.000  0.114   
tulu2_13b   16.224 -23.318  13.275   3.186 -20.472     0.0  0.000  0.000   
tulu2_7b   -12.027  -5.769 -15.529 -15.187 -23.455     0.0  0.000  0.000   

                          
identity     male  white  
model                     
flan_ul2    0.000  0.117  
flant5_xxl  0.129  0.000  
gpt35       0.000  0.000  
gpt4        0.000  0.000  
tulu2_13b   0.001  0.000  
tulu2_7b    0.000  0.000